## MultiTaskLasso: Regresión Lasso para Tareas Múltiples

MultiTaskLasso es una extensión de Lasso Regression diseñada para problemas de aprendizaje supervisado con múltiples tareas o salidas simultáneas. Su principal objetivo es capturar patrones comunes entre estas salidas al penalizar los coeficientes de forma conjunta.

**Ecuación de MultiTaskLasso**
 
El costo que minimiza MultiTaskLasso es:

$
 \text{Costo} = \frac{1}{n} \sum_{i=1}^n ||Y_i - X_i \cdot \beta||_F^2 + \lambda \cdot ||\beta||_{21} 
$

- $||\cdot||_F^2$: Norma de Frobenius, mide el error cuadrático medio para todas las salidas.
 
- $||\beta||_{21}$: Suma de las normas L2 de los coeficientes de cada característica:

$
 ||\beta||_{21} = \sum_{j=1}^p \sqrt{\sum_{k=1}^t \beta_{jk}^2} 
$

Aquí:
 
- $p$: Número de características.
 
- $t$: Número de salidas.
 
- $\lambda$: Hiperparámetro que controla la fuerza de la penalización.


**¿Cuándo usar MultiTaskLasso?**  
1. **Problemas con múltiples salidas:** 
Cuando tienes varias variables objetivo relacionadas y quieres modelarlas simultáneamente.
 
2. **Relaciones entre salidas:** 
Si las salidas comparten características importantes (por ejemplo, predecir diferentes tipos de gastos basados en ingresos, edad, y ubicación), MultiTaskLasso puede capturar esas relaciones.
 
3. **Selección de características compartidas:** 
Si solo unas pocas características son relevantes para todas las salidas, MultiTaskLasso puede identificarlas automáticamente.


In [19]:
import plotly.graph_objects as go
from sklearn.linear_model import MultiTaskLasso
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Crear un conjunto de datos con múltiples salidas
X, Y = make_regression(n_samples=1000, n_features=10, n_targets=3, noise=0.5, random_state=42)

# Dividir los datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 10)
(200, 10)
(800, 3)
(200, 3)


In [20]:
# crear modelo de regresion lineal simple con el conjunto de entrenamiento
regression = MultiTaskLasso(alpha=1.0)
regression.fit(x_train,y_train)
regression

MultiTaskLasso()

In [21]:
# Predecir el conjunto de testeo, solo toma los valores x para predecir los valores de y_test
y_pred = regression.predict(x_test)
print(f"Mean squared error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"""Max {y_test.max()}|Min {y_test.min()}|Mean {y_test.mean()}\nMax {y_pred.max()}|Min {y_pred.min()}|Mean {y_pred.mean()}""")
print(regression.coef_)
print(f"Coeficiente de determinación (R^2): {regression.score(x_test, y_test):.2f}")
# y_pred[:,0]

Mean squared error: 3.97
Max 596.8287279453774|Min -526.1551298652129|Mean 12.972312048870512
Max 590.5123543528198|Min -520.5672304361491|Mean 12.81372035533918
[[51.1428968  72.1852864   9.3187397  31.17343096 20.8255816  42.24239146
  63.17371291 75.6632265  33.30070274 60.840157  ]
 [80.86332167 74.07570474 57.32749265 10.03699888 51.64831954 58.64948958
  17.02649309 73.84124084 74.37140149 14.9443934 ]
 [71.67342264 39.3834737   5.15668336  6.89385135 91.32960585 72.65063431
  67.41594083 24.42367462 29.10598711 82.62882597]]
Coeficiente de determinación (R^2): 1.00


In [24]:
fig = go.Figure([
    go.Scatter(x=x_train[:,0].squeeze(), y=y_train[:,0], name='Train', mode='markers'),
    go.Scatter(x=x_test[:,0].squeeze(), y=y_test[:,0], name='Test', mode='markers'
                ,marker=dict(size=8)),
    go.Scatter(x=x_test[:,0].squeeze(), y=y_pred[:,0], name='Prediction', mode='markers',
                marker=dict(size=4))
])
fig.update_layout(title='Diabetes dataset', xaxis_title='X', yaxis_title='Y', template='plotly_dark')
fig.show()